In [61]:
# General Imports
import datetime
import os
import random
import requests
import re
import string
from collections import defaultdict
import json

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
#Save a json file with api keys for tmdb, openai, pinecone, and postgres username/password
api_keys = json.load(open("./Credentials/api_keys.json"))

## Misc functions

In [3]:
import Levenshtein
def closest_key(query, dictionary):
    closest_key = min(dictionary.keys(), key=lambda k: Levenshtein.distance(query, k))
    return dictionary[closest_key]
def find_repeated_first_elements(tuples_list):
    element_dict = defaultdict(list)
    repeated_elements = {}
    for t in tuples_list:
        element_dict[t[0]].append(t)
    
    for key, value in element_dict.items():
        if len(value) > 1:
            repeated_elements[key] = value
    
    return repeated_elements

# First Functionality - semantic search

## Scrape from Wikipedia

In [4]:
#We get a list of all movies from wikipedia which serves as a source of information that will be utilized in the semantic
#search aspect of this tool

In [5]:
all_urls = [
    "https://en.wikipedia.org/wiki/List_of_films:_A",
    "https://en.wikipedia.org/wiki/List_of_films:_B",
    "https://en.wikipedia.org/wiki/List_of_films:_C",
    "https://en.wikipedia.org/wiki/List_of_films:_D",
    "https://en.wikipedia.org/wiki/List_of_films:_E",
    "https://en.wikipedia.org/wiki/List_of_films:_F",
    "https://en.wikipedia.org/wiki/List_of_films:_G",
    "https://en.wikipedia.org/wiki/List_of_films:_H",
    "https://en.wikipedia.org/wiki/List_of_films:_I",
    "https://en.wikipedia.org/wiki/List_of_films:_J%E2%80%93K",
    "https://en.wikipedia.org/wiki/List_of_films:_L",
    "https://en.wikipedia.org/wiki/List_of_films:_M",
    "https://en.wikipedia.org/wiki/List_of_films:_N%E2%80%93O",
    "https://en.wikipedia.org/wiki/List_of_films:_P",
    "https://en.wikipedia.org/wiki/List_of_films:_Q%E2%80%93R",
    "https://en.wikipedia.org/wiki/List_of_films:_S",
    "https://en.wikipedia.org/wiki/List_of_films:_T",
    "https://en.wikipedia.org/wiki/List_of_films:_U%E2%80%93W",
    "https://en.wikipedia.org/wiki/List_of_films:_X%E2%80%93Z"
]
def get_soup(url):
    response = requests.get(url)
    page_content = response.content
    soup = BeautifulSoup(page_content, 'html.parser')
    return soup
def get_all_movies(url):
    soup = get_soup(url)
    movie_info = []

    for li in soup.find_all('li'):
        a = li.find('a')
        if a:
            title = a.get('title')
            href = a.get('href')

            year_match = re.search(r'\((\d{4})\)', li.text)
            year = year_match.group(1) if year_match else 'Unknown'
            if year == "Unknown":
                continue
            try:
                title = re.sub(r' \([^)]*(film|movie)[^)]*\)', '', title)
            except:
                continue
            movie_info.append((title, href, year))
    
    for li in soup.find_all('li'):
        main_title = li.find('i').text if li.find('i') else ''

        a_tags = li.find_all('a')
        for a in a_tags:
            title = a.get('title', main_title)
            href = a.get('href', '')

            year_match = re.search(r'(\d{4})', a.text)
            year = year_match.group(1) if year_match else 'Unknown'
            if year == "Unknown":
                continue
            title = re.sub(r' \([^)]*(film|movie)[^)]*\)', '', title)
            movie_info.append((title, href, year))
    return list(set(movie_info))

In [6]:
all_movies = []
for url in all_urls:
    all_movies = all_movies + get_all_movies(url)

In [7]:
random.shuffle(all_movies)

#### In all_movies, there's an entry  ('Program (The Animatrix)', '/wiki/Program_(The_Animatrix)', '2003')], but there's no actual movie by that name. Program is an episode


In [8]:
d_movies = get_all_movies("https://en.wikipedia.org/wiki/List_of_films:_D")

In [9]:
d_subset = d_movies[800:850]

## Obtaining snippets

In [10]:
#We scrape the particular wikipedia pages and scrape a "snippet" chunk of text to reference 

In [11]:
def chunk_text(text, n, overlap):
    if overlap >= n:
        return "Overlap must be smaller than chunk size"
    length = len(text)
    start = 0
    chunks = []
    while start < length:
        end = min(start + n, length)
        chunks.append(text[start:end])
        start += n - overlap
        
    return chunks

def get_page_content(href):
    soup = get_soup(f'https://en.wikipedia.org/{href}')
    article_container = soup.find('div', {'id': 'mw-content-text'})
    article_text = ''
    for paragraph in article_container.find_all('p'):
        article_text += paragraph.text
    infobox = soup.find('table', {'class': 'infobox'})

    infobox_text = ""

    if infobox:
        for row in infobox.find_all('tr'):
            # Extract the header if available
            th = row.find('th')
            if th:
                infobox_text += th.get_text() + "\n"

            # Extract the data in the row
            td = row.find('td')
            if td:
                infobox_text += td.get_text() + "\n"
    article_text += infobox_text
    div_tag = soup.find('div', {'class': 'div-col'})
    ul_tag = div_tag.find('ul') if div_tag else None
    list_text = ""
    if ul_tag:
        for li_tag in ul_tag.find_all('li'):
            list_text += li_tag.get_text()
    article_text += list_text
    return article_text

def get_chunked_embeddings(href, embedding_model):
    page_content = get_page_content(href)
    embeddings = embedding_model.encode(chunker(page_content))
    return embeddings

## Embeddings

In [12]:
#Embedding models allow us to encode our sentences into vectors, particularly we will be using a combination of
#sparse and dense embedding models for fine grained semantic search

In [13]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from splade.models.transformer_rep import Splade
import torch
from transformers import AutoTokenizer

C:\Users\pleasework\PycharmProjects\MovieAnalysis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
#intutive embedding test
test_model = SentenceTransformer('all-mpnet-base-v2',device=device)
query_embedding = test_model.encode("thriller, crime")
passage_embedding = test_model.encode("thriller, drama")
passage_embedding2 = test_model.encode("thriller, comedy")
print(util.dot_score(query_embedding, passage_embedding))
print(util.dot_score(query_embedding, passage_embedding2))

tensor([[0.8859]])
tensor([[0.8596]])


In [16]:
dense_model = SentenceTransformer(
    'msmarco-bert-base-dot-v5',
    device=device
)
dense_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [17]:
sparse_model_id = 'naver/splade-cocondenser-ensembledistil'

sparse_model = Splade(sparse_model_id, agg='max')
sparse_model.to(device)
sparse_model.eval()

In [18]:
tokenizer = AutoTokenizer.from_pretrained(sparse_model_id)

## Scrape TMDB

In [19]:
#This was the first site that I found that gave metadata information for movies, letterbox may be more comprehensive
#but this information is suitable for this first functionality

In [20]:
import tmdbsimple as tmdb
tmdb.API_KEY = api_keys["TMDB_API_KEY"]

In [21]:
#genre codes tmdb uses
movie_genre_dict = {28:"Action",12:"Adventure",16:"Animation", 35:"Comedy", 80:"Crime", 99:"Documentary",18:"Drama", 10751:"Family",14:"Fantasy",36:"History",27:"Horror",10402:"Music",9648:"Mystery",10749:"Romance",878:"Science Fiction",10770:"TV Movie", 53:"Thriller", 10752:"War",37:"Western"}
def id2name(id_list):
    return [movie_genre_dict[idz] for idz in id_list]
#language codes tmdb uses
language_codes = {"af":"Afrikaans", "sq":"Albanian", "ar":"Arabic", "eu": "Basque", "bg":"Bulgarian",
                  "ca":"Catalan", "cn":"Chinese", "zh":"Chinese", "hr":"Croatian", "cs":"Czech", "da": "Danish", "nl":"Dutch",
                  "en":"English", "et": "Estonian", "fi":"Finnish", "fr":"French", "de": "German", "el": "Greek",
                  "he":"Hebrew", "hi": "Hindi", "hu":"Hungarian", "is":"Icelandic", "in": "Indonesian", "it": "Italian",
                  "ja": "Japanese", "ko": "Korean", "lv": "Latvian", "lt":"Lithuanian", "mk": "Macedonian", "ms": "malay",
                  "no": "Norwegian", "pl":"Polish", "pt": "Portugese", "rm": "Raeto-Romance", "ro":"Romanian", "ru":"Russian",
                  "sr": "Serbian", "sl": "Slovak", "sk": "Slovenian", "es": "Spanish", "sv": "Swedish", "tr": "Thai",
                  "th": "Turkish", "vi": "Vietnamese"}
def modify_metadata(metadata):
    keys_to_delete = []
    for key in metadata.keys():
        if metadata[key] is None:
            keys_to_delete.append(key)

    for key in keys_to_delete:
        del metadata[key]
    metadata["genre_ids"] = id2name(metadata["genre_ids"])
    return metadata

In [22]:
search = tmdb.Search()

In [23]:
unembedded_data = {}
all_data = []
for index, movie in tqdm(enumerate(d_subset), total=len(d_subset)):
    #How to make a search using TMDB
    tmdb_responses = search.movie(query=movie[0])["results"]
    if not len(tmdb_responses):
        print(f"{movie[0]} was not found in db")
        continue
    metadata = None
    for result in tmdb_responses:
        if result["release_date"][:4] == movie[2]:
            metadata = result
    if metadata == None:
        continue
    metadata = modify_metadata(metadata)
    page_content = get_page_content(movie[1])
    chunks = chunk_text(page_content, 384, 20)
    chunked_embeddings = dense_model.encode(chunks)
    input_ids = tokenizer(
        chunks, return_tensors='pt',
        padding=True, truncation=True
    )
    with torch.no_grad():
        sparse_vecs = sparse_model(
            d_kwargs=input_ids.to(device)
        )['d_rep'].squeeze()
    for index, embedding in enumerate(chunked_embeddings):
        new_id = f"{metadata['id']}-{index}"
        dense_embedding = embedding.tolist()
        sparse_vec = sparse_vecs[index]
        indices = sparse_vec.nonzero().squeeze().cpu().tolist()  # positions
        values = sparse_vec[indices].cpu().tolist()  # weights/scores
        # build sparse values dictionary
        sparse_values = {
            "indices": indices,
            "values": values
        }
        all_data.append(
            {"id":new_id,
             "values":dense_embedding,
             "sparse_values": sparse_values,
             "metadata":metadata}
        )
        unembedded_data[new_id] = chunks[index]

  2%|██▍                                                                                                                         | 1/50 [00:00<00:07,  6.84it/s]C:\Users\pleasework\PycharmProjects\MovieAnalysis\venv\lib\site-packages\torch\amp\autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:00<00:00,  6.02s/it]


## Use Pinecone to host and search snippets

In [24]:
import pinecone
pinecone.init(api_key = api_keys["PINECONE_API_KEY"], environment="gcp-starter")

In [25]:
index_name = "movie-finder"

In [26]:
## Create index if it hasn't been created yet

In [27]:
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="dotproduct",
    )

In [28]:
index = pinecone.GRPCIndex(index_name)

In [29]:
def batch_list(input_list, n=100):
    for i in range(0, len(input_list), n):
        yield input_list[i:i + n]
counter = 0
for batch in batch_list(all_data):
    print(counter)
    index.upsert(vectors=batch)
    counter += 1

0
1
2
3
4
5
6
7


## Semantic search with GPT for reasoning

In [30]:
import openai
openai.api_key = api_keys["OPENAI_API_KEY"]

In [31]:
def encode(text: str):
    dense_vec = dense_model.encode(text).tolist()
    input_ids = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        sparse_vec = sparse_model(
            d_kwargs=input_ids.to(device)
        )['d_rep'].squeeze()
    indices = sparse_vec.nonzero().squeeze().cpu().tolist()
    values = sparse_vec[indices].cpu().tolist()
    sparse_dict = {"indices": indices, "values": values}
    return dense_vec, sparse_dict

In [32]:
def get_context(ids):
    result = []
    for id, metadata in ids:
        result.append(unembedded_data[id] + metadata)
    return "\n".join(result)

In [33]:
def format_metadata(metadata):
    return f"==film_title: {metadata['title']}, genre(s): {metadata['genre_ids']}, original language: {language_codes[metadata['original_language']]}, release_date: {metadata['release_date']}"

In [34]:
def get_response(question, k=5):
    dense, sparse = encode(question)
    xc = index.query(
        vector=dense,
        sparse_vector=sparse,
        top_k=k, 
        include_metadata=True
    )
    response_ids_metadata = [(match["id"], format_metadata(match["metadata"])) for match in xc["matches"]]
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0613",
      messages=[
        {"role": "user", "content": f"Using only the following context:\n{get_context(response_ids_metadata)}\nAnswer this question: {question}"}
        ]
    )
    print(response["choices"][0]["message"]["content"])

In [35]:
get_response("movie with undercover cop and undercover mafia")

KeyError: '860317-1'

In [ ]:
get_response("movie where two guys save a gym")

In [ ]:
get_response("movie where monkey goes to the moon.")

# Second Functionality - text2SQL

## Scrape Letterbox

In [ ]:
# The goal of this aspect of the project is to enable arbitrary queries of the type:
    #I want (genre) (language), (release date), (runtime) films that are similar to x,y, ... in (genre),(description),(cast),(crew)
# This involves first scraping all data related to cast, genre, themes, etc. films from letterbox and uploading to a db
# then use GPT to convert a natural language query into a SQL query

In [68]:
## Some details of the movies were not included in their page

def get_themes(soup):
    sections = soup.find_all('section', {'class': 'genre-group'})
    section_texts = []
    for section in sections:
        section_text = section.text.strip()
        section_texts.append(section_text)
    return section_texts
def get_cast(soup):
    cast_list_div = soup.find('div', {'class': 'cast-list text-sluglist'})
    cast_names = []
    if cast_list_div:
        actor_tags = cast_list_div.find_all('a', {'class': 'text-slug tooltip'})
        for tag in actor_tags:
            cast_names.append(tag.text.lower())
    return cast_names
def get_genres(soup):
    genres_div = soup.find('div', {'class': 'text-sluglist capitalize'})
    genre_names = []
    if genres_div:
        genre_tags = genres_div.find_all('a', {'class': 'text-slug'})
        for tag in genre_tags:
            genre_names.append(tag.text)
    return genre_names
def get_studios(soup):
    h3_studios = soup.find('h3', text=lambda t: 'Studios' in t if t else False)
    studio_names = []
    if h3_studios:
        studios_div = h3_studios.find_next('div', {'class': 'text-sluglist'})
        if studios_div:
            studio_tags = studios_div.find_all('a', {'class': 'text-slug'})
            for tag in studio_tags:
                studio_names.append(tag.text)
    return studio_names

def get_original_language(soup):
    h3_language = soup.find('h3', text=lambda t: 'Original Language' in t if t else False)
    original_languages = []
    if h3_language:
        language_div = h3_language.find_next('div', {'class': 'text-sluglist'})

        if language_div:
            language_tags = language_div.find_all('a', {'class': 'text-slug'})
            for tag in language_tags:
                original_languages.append(tag.text)
    return original_languages[0].lower() if len(original_languages) else ""

def get_similar(soup):
    target_links = []
    div_tags = soup.find_all('div', {'class': 'really-lazy-load'})
    for div in div_tags:
        link = div.get('data-target-link')
        if link:
            target_links.append(link.replace("/film/", "").strip("/"))
    return target_links

def get_release_year(soup):
    try:
        year = soup.select_one('section#featured-film-header small.number a').text
    except AttributeError:
        print('Release year was not available on the page')
        year = ''
        pass
    return year

def get_runtime(soup):
    p_tag = soup.find('p', {'class': ['text-link', 'text-footer']})
    if p_tag:
        text_content = p_tag.text
        match = re.search(r'(\d+)\s*mins', text_content)
        if match:
            mins = int(match.group(1))   
            return mins
        
def get_watched_stats(soup):
    try:
        watched = soup.find('li', {'class': 'js-route-watches'}).find('a', {'class': 'tooltip'})['title'].split()[0].replace(',', '')
    except:
        watched = 0
    try:
        fans = soup.find('li', {'class': 'js-route-fans'}).find('a', {'class': 'tooltip'})['title'].split()[0].replace(',', '')
    except:
        fans = 0
    try:
        likes = soup.find('li', {'class': 'js-route-likes'}).find('a', {'class': 'tooltip'})['title'].split()[0].replace(',', '')
    except:
        likes = 0
    try:
        reviews = soup.find('li', {'class': 'js-route-reviews'}).find('a', {'class': 'tooltip'})['title'].split()[0].replace(',', '')
    except:
        reviews = 0
    return watched, fans, likes, reviews

def get_crew(soup):
    roles_and_names = {}
    for h3_tag in soup.find_all('h3'):
        role_span = h3_tag.find('span', {'class': 'crewrole -full'})
        if role_span:
            role_full = role_span.text
            name_div = h3_tag.find_next('div', {'class': 'text-sluglist'})
            if name_div:
                name_tags = name_div.find_all('a', {'class': 'text-slug'})
                names = [name_tag.text.lower() for name_tag in name_tags]
                roles_and_names[role_full.lower()] = names
    return roles_and_names

def modify_movie(movie_title):
    return ''.join(char for char in movie_title if char not in string.punctuation).lower().replace(' ', '-')

def find_movie(movie, url_base):
    movie_title = movie[0]
    year = movie[2]
    modified_title = modify_movie(movie_title)
    with_year = f"{modified_title}-{year}"
    soup = get_soup(f"{url_base}{with_year}")
    if "Not Found" in soup.title.string:
        soup = get_soup(f"{url_base}{modified_title}")
        if "Not Found" in soup.title.string:
            return None, None, None
        else:
            validate_year = get_release_year(soup)
            if validate_year == year:
                return soup, modified_title, year
            else:
                return None, None, None
    else:
        return soup,with_year,year

def scrape_all(movie, url_base):
    soup, letterbox_title, year = find_movie(movie, url_base)
    if soup is None:
        return None
    scraped_dict = {}
    scraped_dict["title"] = movie[0]
    scraped_dict["genres"] = get_genres(soup)
    scraped_dict["studios"] = get_studios(soup)
    scraped_dict["original_language"] = get_original_language(soup)
    scraped_dict["release year"] = year
    scraped_dict["runtime"] = get_runtime(soup)
    crew_dict = get_crew(soup)
    crew_dict["actor"] = get_cast(soup)
    scraped_dict["crew"] = crew_dict
    scraped_dict["themes"] = get_themes(get_soup(f"{url_base}{letterbox_title}/themes/"))
    all_similar = get_similar(get_soup(f"{url_base}{letterbox_title}/similar"))
    scraped_dict["similar"] = [film for film in all_similar if film != letterbox_title]
    stats = get_watched_stats(get_soup(f"{url_base}{letterbox_title}/members"))
    scraped_dict["num_watched"] = stats[0]
    scraped_dict["num_fans"] = stats[1]
    scraped_dict["num_liked"] = stats[2]
    scraped_dict["num_reviewed"] = stats[3]
    return {letterbox_title: scraped_dict}    

In [70]:
full_dict={}

In [65]:
## Have this loop to make randomized user data

In [71]:
## Troubleshooting scraping
scrapedList=[]

In [72]:
start = datetime.datetime.now()
url_base = "https://letterboxd.com/film/"
for movie in tqdm(all_movies[:5000]):
    scrapedList.append(movie)
    if movie not in full_dict:
        full_dict[movie] = scrape_all(movie, url_base)
    else:
        pass
print(datetime.datetime.now() - start)

  0%|                                                                                                                                  | 0/5000 [00:00<?, ?it/s]C:\Users\pleasework\AppData\Local\Temp\ipykernel_14092\4138716162.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  h3_studios = soup.find('h3', text=lambda t: 'Studios' in t if t else False)
C:\Users\pleasework\AppData\Local\Temp\ipykernel_14092\4138716162.py:38: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  h3_language = soup.find('h3', text=lambda t: 'Original Language' in t if t else False)
  0%|▏                                                                                                                       | 8/5000 [00:13<2:07:08,  1.53s/it]

Release year was not available on the page


  4%|████▊                                                                                                                 | 204/5000 [07:07<2:10:02,  1.63s/it]

Release year was not available on the page


 10%|████████████▎                                                                                                         | 523/5000 [18:18<2:13:11,  1.78s/it]

...

Release year was not available on the page


 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 4853/5000 [3:00:39<02:43,  1.11s/it]

Release year was not available on the page


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [3:06:06<00:00,  2.23s/it]

3:06:06.642549


In [76]:
## credit to user Tms91 from the following thread: https://stackoverflow.com/questions/7001606/json-serialize-a-dictionary-with-tuples-as-key

def json_dumps_dict_having_tuple_as_key(dict_having_tuple_as_key):
    if not isinstance(dict_having_tuple_as_key, dict):
        raise Exception('Error using json_dumps_dict_having_tuple_as_key: The input variable is not a dictionary.')  
    list_of_dicts_having_key_and_value_as_keys = [{'key': k, 'value': v} for k, v in dict_having_tuple_as_key.items()]
    json_array_having_key_and_value_as_keys = json.dumps(list_of_dicts_having_key_and_value_as_keys)
    return json_array_having_key_and_value_as_keys

def json_loads_dictionary_split_into_key_and_value_as_keys_and_underwent_json_dumps(json_array_having_key_and_value_as_keys):
    list_of_dicts_having_key_and_value_as_keys = json.loads(json_array_having_key_and_value_as_keys)
    if not all(['key' in diz for diz in list_of_dicts_having_key_and_value_as_keys]) and all(['value' in diz for diz in list_of_dicts_having_key_and_value_as_keys]):
        raise Exception('Error using json_loads_dictionary_split_into_key_and_value_as_keys_and_underwent_json_dumps: at least one dictionary in list_of_dicts_having_key_and_value_as_keys ismissing key "key" or key "value".')
    dict_having_tuple_as_key = {}
    for dict_having_key_and_value_as_keys in list_of_dicts_having_key_and_value_as_keys:
        dict_having_tuple_as_key[ tuple(dict_having_key_and_value_as_keys['key']) ] = dict_having_key_and_value_as_keys['value']
    return dict_having_tuple_as_key

In [77]:
output = json_dumps_dict_having_tuple_as_key(full_dict)

In [81]:
with open("fulldict.json", "w") as outfile:
    json.dump(output, outfile)

In [82]:
with open('fulldict.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
 
print(json_object)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
import random
def generateUsers(numUsers, listOfUsers, fullMovieDict):
    userDict = {}
    for i in range(i, numUsers):
        listOfMovies = []
        for j in range(j,10):
            listOfMovies.append(random.choice(list(fullMovieDict.items())))
        userDict[listOfUsers[i]] = listOfMovies
    return userDict

In [ ]:
## Given your top 10 movies, return the most frequently appearing "similar movies" 

In [ ]:
from functools import reduce
import operator
def getitem(d,key):
    return reduce(operator.getitem, key, d)

In [ ]:
check = []
counter = 0
for key in full_dict:
    try:
        # Since the nested key name is somewhat variable and the positions are always the same, jury rigged a simple solution 
        check.append(getitem(full_dict[key][list(full_dict[key].keys())[0]], ['crew', 'director'])[0])
        counter += 1
    except AttributeError:
        print('There was a problem with the way the data was scraped')
        check.append('user'+str(counter))
        counter += 1

In [ ]:
dict_errors = []
similar_movies_dict = {}
for key in full_dict:
    try:
        similar_movies_dict[key] = full_dict[key][list(full_dict[key].keys())[0]]['similar']
    except KeyError:
        print('Letterboxd probably did not include a list of similar movies for this title')
        pass
    except TypeError:
        # Some urls don't link to an actual title, removing those from the dict
        print('There was an error with the url, there is no individual title by that name')
        dict_errors.append(key)
        pass
for key in dict_errors:
    del full_dict[key]

In [ ]:
# This function literally counts how many times each element shows up in a list. 
# So if we just add the lists of similar movies from the top 10 into one, that should work
from collections import Counter
Counter(sum(similar_movies_dict.values(), [])).most_common(5)

## Upload data to PostgresSQL

In [ ]:
import psycopg2
import json

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres", 
    user=api_keys["POSTGRES_USER"],
    password=api_keys["POSTGRES_PASSWORD"],
    port="5432"
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS movies (
    id SERIAL PRIMARY KEY,
    title VARCHAR(255),
    letterbox_title VARCHAR(255) UNIQUE,
    original_language VARCHAR(50),
    release_year INT,
    runtime INT,
    num_watched INT,
    num_fans INT,
    num_liked INT,
    num_reviewed INT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS genres (
    id SERIAL PRIMARY KEY,
    genre_name VARCHAR(255) UNIQUE
);
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS roles (
    id SERIAL PRIMARY KEY,
    role_name VARCHAR(255) UNIQUE
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS themes (
    id SERIAL PRIMARY KEY,
    theme_name VARCHAR(255) UNIQUE
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS studios (
    id SERIAL PRIMARY KEY,
    studio_name VARCHAR(255) UNIQUE
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS similar_movies (
    movie_id INT,
    similar_movie_id INT,
    PRIMARY KEY (movie_id, similar_movie_id),
    FOREIGN KEY (movie_id) REFERENCES movies (id),
    FOREIGN KEY (similar_movie_id) REFERENCES movies (id)
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS movie_crew (
    movie_id INT,
    role_id INT,
    crew_name VARCHAR(255),
    PRIMARY KEY (movie_id, role_id, crew_name),
    FOREIGN KEY (movie_id) REFERENCES movies (id),
    FOREIGN KEY (role_id) REFERENCES roles (id)
);
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS movie_studios (
    movie_id INT,
    studio_id INT,
    PRIMARY KEY (movie_id, studio_id),
    FOREIGN KEY (movie_id) REFERENCES movies (id),
    FOREIGN KEY (studio_id) REFERENCES studios (id)
);
""")


cur.execute("""
CREATE TABLE IF NOT EXISTS movie_genres (
    movie_id INT,
    genre_id INT,
    PRIMARY KEY (movie_id, genre_id),
    FOREIGN KEY (movie_id) REFERENCES movies (id),
    FOREIGN KEY (genre_id) REFERENCES genres (id)
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS movie_themes (
    movie_id INT,
    theme_id INT,
    PRIMARY KEY (movie_id, theme_id),
    FOREIGN KEY (movie_id) REFERENCES movies (id),
    FOREIGN KEY (theme_id) REFERENCES themes (id)
);
""")

dataset = full_dict

for key, movie_data in dataset.items():
    cur.execute("""
    INSERT INTO movies (title, letterbox_title, original_language, release_year, runtime, num_watched, num_fans, num_liked, num_reviewed)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING id;
    """, (
        movie_data.get('title', None),
        key,
        movie_data.get('original_language', None),
        movie_data.get('release year', None),
        movie_data.get('runtime', None),
        movie_data.get('num_watched', None),
        movie_data.get('num_fans', None),
        movie_data.get('num_liked', None),
        movie_data.get('num_reviewed', None)
    ))

    movie_id = cur.fetchone()[0]

    for genre in movie_data.get('genres', []):
        cur.execute("""
        INSERT INTO genres (genre_name) VALUES (%s) ON CONFLICT (genre_name) DO NOTHING;
        """, (genre,))

        cur.execute("""
        INSERT INTO movie_genres (movie_id, genre_id) SELECT %s, id FROM genres WHERE genre_name = %s;
        """, (movie_id, genre))
        
    for studio in movie_data.get('studios', []):
        cur.execute("""
        INSERT INTO studios (studio_name) VALUES (%s) ON CONFLICT (studio_name) DO NOTHING;
        """, (studio,))

        cur.execute("""
        INSERT INTO movie_studios (movie_id, studio_id) SELECT %s, id FROM studios WHERE studio_name = %s;
        """, (movie_id, studio))
        
    for theme in movie_data.get('themes', []):
        cur.execute("""
        INSERT INTO themes (theme_name) VALUES (%s) ON CONFLICT (theme_name) DO NOTHING;
        """, (theme,))

        cur.execute("""
        INSERT INTO movie_themes (movie_id, theme_id) SELECT %s, id FROM themes WHERE theme_name = %s;
        """, (movie_id, theme))
        
    for role, crew_names in movie_data.get('crew', {}).items():
        cur.execute("""
        INSERT INTO roles (role_name) VALUES (%s) ON CONFLICT (role_name) DO NOTHING;
        """, (role,))

        for crew_name in crew_names:
            cur.execute("""
            INSERT INTO movie_crew (movie_id, role_id, crew_name)
            SELECT %s, id, %s FROM roles WHERE role_name = %s;
            """, (movie_id, crew_name, role))
    
    for similar_movie in movie_data.get('similar', []):
        cur.execute("""
        INSERT INTO similar_movies (movie_id, similar_movie_id)
        SELECT %s, id FROM movies WHERE letterbox_title = %s;
        """, (movie_id, similar_movie))

conn.commit()
cur.close()
conn.close()

In [ ]:
import faiss